In [1]:
!pip install transformers
!pip install accelerate>=0.26.0
!pip install datasets
!pip install -U bitsandbytes
!pip install sae_lens
!pip install huggingface_hub
import json
import os
import datasets
from sae_lens import SAE
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import notebook_login
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 39.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 132.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 166.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 138.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 210.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformer_lens import HookedTransformer

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model = HookedTransformer.from_pretrained(model_name, device="cuda")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


In [3]:
from sae_lens import SAE

sae, cfg_dict, sparsity = SAE.from_pretrained(release="llama-3-8b-it-res-jh", sae_id="blocks.25.hook_resid_post", device="cuda")

cfg.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

sae_weights.safetensors:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/262k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/sae_lens/sae.py:644: UserWarning: norm_scaling_factor not found for llama-3-8b-it-res-jh and blocks.25.hook_resid_post, but normalize_activations is 'expected_average_only_in'. Skipping normalization folding.
  warnings.warn(


In [4]:
# get hook point
hook_point = sae.cfg.hook_name
print(hook_point)

blocks.25.hook_resid_post


In [5]:
from datasets import load_dataset
dataset = load_dataset("gsm8k", "main")

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [6]:
train_questions = [item["question"] for item in dataset["train"]]
test_questions = [item["question"] for item in dataset["test"]]

In [7]:
sv_prompt = " The Golden Gate Bridge"
sv_logits, cache = model.run_with_cache(sv_prompt)
tokens = model.to_tokens(sv_prompt)
print(tokens)

# get the feature activations from our SAE
sv_feature_acts = sae.encode(cache[hook_point])

# get sae_out
sae_out = sae.decode(sv_feature_acts)

# print out the top activations, focus on the indices
print(torch.topk(sv_feature_acts, 3))

tensor([[128000,    578,  18288,  30343,  20467]], device='cuda:0')
torch.return_types.topk(
values=tensor([[[12.1599, 12.1552, 12.0955],
         [ 9.5191,  5.9392,  3.5369],
         [17.2081,  2.9424,  2.5228],
         [ 5.6462,  4.2330,  3.9888],
         [ 6.4601,  2.9334,  2.7729]]], device='cuda:0',
       grad_fn=<TopkBackward0>),
indices=tensor([[[61272,  3179, 28003],
         [38702,   607, 63905],
         [44661, 21229, 37976],
         [ 2999, 23747,  5992],
         [   87, 62317, 23747]]], device='cuda:0'))


In [9]:
last_token_index = sv_feature_acts.shape[1] - 1  

last_token_activations = sv_feature_acts[0, last_token_index, :]

print(f"Shape of last token activations: {last_token_activations.shape}")  

print("Last token's all SAE feature activations:")
print(last_token_activations)

print(torch.topk(last_token_activations, 65536))
print(last_token_activations[:88])

Shape of last token activations: torch.Size([65536])
Last token's all SAE feature activations:
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SliceBackward0>)
torch.return_types.topk(
values=tensor([6.4601, 2.9334, 2.7729,  ..., 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<TopkBackward0>),
indices=tensor([   87, 62317, 23747,  ..., 65533, 65534, 65535], device='cuda:0'))
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.00

In [75]:
import torch
import random
from sae_lens import SAE

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)


N = 800
role_play_prompt = "As a highly qualified mathematics teacher, you excel at solving problems systematically and explaining solutions with clarity. I am your student, eager to learn. Please solve the following problem:"

selected_questions = [random.choice(train_questions) for _ in range(N)]

with_role_prompts = [
    f"{role_play_prompt} {q}"
    for q in selected_questions
]
without_role_prompts = [
    f"{q}"
    for q in selected_questions
]

# with_role_prompts = [
#     f"{role_play_prompt} {q} Please output your final answer (a number) after 'Output: '"
#     for q in selected_questions
# ]
# without_role_prompts = [
#     f"{q} Please output your final answer (a number) after 'Output: '"
#     for q in selected_questions
# ]


# print(with_role_prompts)
# print(without_role_prompts)


with_role_feature_acts_list = []
without_role_feature_acts_list = []


for i in tqdm(range(N), desc="Processing Prompts", unit="pair"):
    
    with torch.no_grad():  
       
        with_role_prompt = with_role_prompts[i]
        without_role_prompt = without_role_prompts[i]

        with_role_logits, with_role_cache = model.run_with_cache(with_role_prompt, prepend_bos=True)
        without_role_logits, without_role_cache = model.run_with_cache(without_role_prompt, prepend_bos=True)

        with_role_feature_acts = sae.encode(with_role_cache[hook_point])
        without_role_feature_acts = sae.encode(without_role_cache[hook_point])

        with_role_last_token_index = with_role_feature_acts.shape[1] - 1
        without_role_last_token_index = without_role_feature_acts.shape[1] - 1

        with_role_last_token_activations = with_role_feature_acts[0, with_role_last_token_index, :]
        without_role_last_token_activations = without_role_feature_acts[0, without_role_last_token_index, :]

        with_role_feature_acts_list.append(with_role_last_token_activations)
        without_role_feature_acts_list.append(without_role_last_token_activations)

    
    del with_role_cache, without_role_cache, with_role_feature_acts, without_role_feature_acts
    del with_role_logits, without_role_logits
    torch.cuda.empty_cache()  
    torch.cuda.ipc_collect()



Processing Prompts: 100%|██████████| 800/800 [04:17<00:00,  3.11pair/s]


In [77]:
with_role_feature_acts = torch.stack(with_role_feature_acts_list)  # (N, feature_dim)
without_role_feature_acts = torch.stack(without_role_feature_acts_list)  # (N, feature_dim)

delta_h = ((with_role_feature_acts > 0).float() - (without_role_feature_acts > 0).float())

sensitivity_score = (delta_h > 0).float().mean(dim=0)  
k = 15
top_k_indices = torch.topk(sensitivity_score, k=k).indices
top_k_features = sensitivity_score[top_k_indices]


print(f"Top-{k} Role-Play Influenced Features (Indices & Sensitivity Scores):")
for idx, score in zip(top_k_indices.tolist(), top_k_features.tolist()):
    print(f"Feature {idx}: Sensitivity {score:.4f}")

Top-15 Role-Play Influenced Features (Indices & Sensitivity Scores):
Feature 2585: Sensitivity 0.8475
Feature 26525: Sensitivity 0.7800
Feature 34989: Sensitivity 0.6850
Feature 17341: Sensitivity 0.6137
Feature 314: Sensitivity 0.5650
Feature 9300: Sensitivity 0.5462
Feature 33603: Sensitivity 0.4175
Feature 19101: Sensitivity 0.4112
Feature 57830: Sensitivity 0.4038
Feature 44566: Sensitivity 0.3887
Feature 24563: Sensitivity 0.3237
Feature 21231: Sensitivity 0.3150
Feature 25834: Sensitivity 0.2788
Feature 19121: Sensitivity 0.2687
Feature 12705: Sensitivity 0.2412


In [78]:
top_k_activations = with_role_feature_acts[:, top_k_indices]  # (N, k)

mean_activation = top_k_activations.mean(dim=0)  # (k,)

std_activation = top_k_activations.std(dim=0)  # (k,)

β = 1.0  

steering_strengths = mean_activation + β * std_activation  # (k,)

steering_vectors = sae.W_dec[top_k_indices]  

steering_shift = (steering_strengths[:, None] * steering_vectors).sum(dim=0)  # (feature_dim,)


In [79]:
def steering_hook(resid_pre, hook):
    
    if resid_pre.shape[1] == 1:
        return

    position = resid_pre.shape[1] - 1 
    if steering_on:
        resid_pre[:, position, :] += steering_shift  


def hooked_generate(prompt_batch, fwd_hooks=[], seed=None):
    
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            input=tokenized,
            max_new_tokens = 512,
            do_sample=True
        )
    return result


In [80]:
from tqdm import tqdm

def run_generate(example_prompt):
    
    model.reset_hooks()
    editing_hooks = [(f"blocks.25.hook_resid_post", steering_hook)]
    res = hooked_generate(
        [example_prompt], editing_hooks, seed=None
    )

    res_str = model.to_string(res[:, 1:])  
    return res_str[0]


In [81]:
steering_on = True

for i in tqdm(range(len(test_questions)), desc="Generating Responses", unit="prompt"):
    example_prompt = test_questions[i] 
    response = run_generate(example_prompt)  
      
    print(f"Question {i+1}: {example_prompt}")  
    print(f"Response {i+1}: {response}")  

    del response
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()


Generating Responses:   0%|          | 0/1319 [00:00<?, ?prompt/s]

  0%|          | 0/512 [00:00<?, ?it/s]

Generating Responses:   0%|          | 1/1319 [00:20<7:36:28, 20.78s/prompt]

Question 1: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Response 1: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Originally, she has 16 eggs, then she eats 3, and then bakes with 4 more. So she will have 16 - 3 - 4 eggs left. That's 9. So she sells 9 eggs.
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmer

  0%|          | 0/512 [00:00<?, ?it/s]

...

Generating Responses:   1%|▏         | 19/1319 [05:56<6:30:05, 18.00s/prompt]

Question 19: Claire makes a 3 egg omelet every morning for breakfast.  How many dozens of eggs will she eat in 4 weeks?
Response 19: Claire makes a 3 egg omelet every morning for breakfast.  How many dozens of eggs will she eat in 4 weeks?  Each week has 7 days, and she eats 3 eggs per day.  That is 21 eggs per week.  So in 4 weeks she will eat 4 * 21 = 84 eggs.  84 eggs divided by 12 (the number of eggs per dozen) is $\frac{84}{12} = \boxed{7}$ dozens.
Final Answer: The final answer is 7. I hope it is correct.्रदeating 3 eggs per day.

Claire makes a 3 egg omelet every morning for breakfast.  How many dozens of eggs will she eat in 4 weeks?  Each week has 7 days, and she eats 3 eggs per day.  That is 21 eggs per week.  So in 4 weeks she will eat 4 * 21 = 84 eggs.  84 eggs divided by 12 (the number of eggs per dozen) is $\frac{84}{12} = \boxed{7}$ dozens.
Final Answer: The final answer is 7. I hope it is correct.cstdlibeating 3 eggs per day.

Claire makes a 3 egg omelet every morning f

  0%|          | 0/512 [00:00<?, ?it/s]

Generating Responses:   1%|▏         | 19/1319 [05:57<6:47:37, 18.81s/prompt]


KeyboardInterrupt: 

In [82]:
steering_on = False

for i in tqdm(range(len(test_questions)), desc="Generating Responses", unit="prompt"):
    example_prompt = test_questions[i] 
    response = run_generate(example_prompt)  
      
    print(f"Question {i+1}: {example_prompt}")  
    print(f"Response {i+1}: {response}")  

    del response
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

Generating Responses:   0%|          | 0/1319 [00:00<?, ?prompt/s]

  0%|          | 0/512 [00:00<?, ?it/s]

Generating Responses:   0%|          | 1/1319 [00:20<7:34:05, 20.67s/prompt]

Question 1: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Response 1: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? First, 16 - 3 - 4 = 9. This means that she has 9 eggs left to sell at the farmers' market daily. 9 * 2 = 18. So she makes $18 dollars every day at the farmers' market.
Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make

  0%|          | 0/512 [00:00<?, ?it/s]

...

Generating Responses:   1%|▏         | 19/1319 [06:00<5:34:24, 15.43s/prompt]

Question 19: Claire makes a 3 egg omelet every morning for breakfast.  How many dozens of eggs will she eat in 4 weeks?
Response 19: Claire makes a 3 egg omelet every morning for breakfast.  How many dozens of eggs will she eat in 4 weeks?  Each dozen eggs is 12.

Claire eats 3 eggs every morning.  In one week, she'll eat 7 eggs (3 a day).  In 4 weeks, she'll eat 4 * 7 = 28.  28 eggs is $\frac{28}{12}$ dozens of eggs.  Simplifying this, we get $\boxed{7/3}$ dozens of eggs.

Final Answer: The final answer is 7/3. I hope it is correct.<|eot_id|>


  0%|          | 0/512 [00:00<?, ?it/s]

Generating Responses:   1%|▏         | 19/1319 [06:08<7:00:20, 19.40s/prompt]


KeyboardInterrupt: 